#### <span style="color:red">Note: this notebook requires Python3 kernel</span>
#### Here are some short scripts to preforme all sorts of analysis over our output data

In [152]:
import os
user='hagailevi'# 'maibendayan'
os.chdir(f'/specific/elkon/{user}/PRS/codebase')
from constants import *
import pandas as pd

#### PRS parameters and directories

In [155]:
discoveries=["bcac_onco_eas1-5pcs", "bcac_onco_eur-5pcs"] # ["bcac_onco_afr_no_1200_usa-5pcs", "bcac_onco_eur-5pcs"]
### make sure to sort the discoveries lexicographically! ###
gwas_pops=["EAS", "EUR"] # ["AFR", "EUR"]

pheno="pheno"
target="bcac_onco_eas2"
imp="impX"

method="pt3"

metric='or.all'
rank_by='or'

In [154]:
# cross validation parameters
n_folds=3
n_repetitions=4
base_rep=102 # n_repetitions+99

In [136]:
inner_test_name='validation.multi'
test_name='test.multi'

In [137]:
mono_discovery0=discoveries[0]
mono_discovery1=discoveries[1]
multi_discovery="+".join(discoveries)

folds=3
reps=4

#### Functions

Create average file of outer executions (and optinally saving the results)

In [24]:
# OPTION 1: AVG test.statistics over all reps

def create_mono_AVG_stat_file(discoveries,target,imp,test_name,n_repetitions,n_folds,base_rep, method,save_file=False):
    mono_avg_dfs=[]
    for discovery in discoveries:
        mono_avg_df=pd.DataFrame()
        for cur_rep in range(1, n_repetitions+1):
            prs_rep_path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/rep_'+str(base_rep)+'_'+str(cur_rep)+'/'+'prs.cv.'+method+'___'
            file_name=prs_rep_path+str(n_folds)+'_'+test_name+'.statistics_by_'+rank_by+'.tsv'
            df_stat=pd.read_csv(file_name, sep='\t')
            df_stat.set_index('hp', inplace = True)
            # sum the results
            mono_avg_df=mono_avg_df.add(df_stat, fill_value=0)
            
        # create the average statistics file:
        mono_avg_df=mono_avg_df/n_repetitions

#         print(mono_avg_df)
        mono_avg_dfs.append(mono_avg_df)
        if save_file:
            output_path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/prs.cv.'+method+'___'
            mono_avg_df.to_csv(output_path+'avg_'+str(n_repetitions)+'_'+test_name+'.statistics_by_'+rank_by+'.tsv',mode="w", sep='\t')
            print ("the next file was created: "+output_path+'avg_'+str(n_repetitions)+'_'+test_name+'.statistics_by_'+rank_by+'.tsv')

    return mono_avg_dfs

In [218]:
# test_name='test'
# dfs=create_mono_AVG_stat_file(discoveries,target,imp,test_name,n_repetitions,n_folds,base_rep, method,save_file=False)
# for df in dfs:
#     display(df[["or.all", "or.rank"]])

Concatenate all reps files into a single DF (and optinally saving the results)

In [11]:
# OPTION 2: concatinated test.statistics over all reps

def create_outer_stat_file(discovery,target,imp,test_name,n_repetitions,n_folds, base_rep, method,save_file=False, rank_by=""):
# for discovery in discoveries:
    
    metric_suffix=""
    if rank_by:
        metric_suffix=f'_by_{rank_by}'

    mono_outer_all_df=pd.DataFrame()
    for cur_rep in range(1, n_repetitions+1):
        prs_rep_path=f'{PRSS_PATH}{discovery}_{target}/{imp}/rep_{str(base_rep)}_{str(cur_rep)}/prs.cv.{method}___'
        file_name=prs_rep_path+str(n_folds)+'_'+test_name+'.statistics'+metric_suffix+'.tsv'
        df_stat=pd.read_csv(file_name, sep='\t')
        # df_stat.set_index('hp', inplace = True)
        df_stat['rep']=cur_rep
        df_stat[f'{metric}.outer.rank']=len(df_stat[metric])-df_stat[metric].rank()+1
        
        # sum the results
        mono_outer_all_df=pd.concat([mono_outer_all_df,df_stat])

    if save_file:
        output_path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/prs.cv.'+method+'___'
        mono_outer_all_df.to_csv(output_path+'all_'+str(n_repetitions)+'_'+test_name+'.statistics'+param_suffix+'.tsv',mode="w", sep='\t')
        print(mono_outer_all_df)
        print ("the next file was created: "+output_path+'all_'+str(n_repetitions)+'_'+test_name+'.statistics'+param_suffix+'.tsv')

    return mono_outer_all_df

In [180]:
# test_name='test'
# create_outer_stat_file(discoveries,target,imp,test_name,n_repetitions,n_folds,base_rep, method,save_file=False)

Average of statistics results w/o using the multi step (mono-ethnic only)

In [141]:
# Do not forget to run this first!
test_name='test'

In [142]:
# non-EUR
df=create_outer_stat_file(discoveries[0],target,imp,test_name,n_repetitions,n_folds,base_rep, method,save_file=False) 
df[['hp', metric, f'{metric}.outer.rank', 'rep']].groupby('hp')[[metric]].mean()

,or.all
hp,
5.000000e-08,1.243448
1.000000e-07,1.243448
1.000000e-06,1.243448
1.000000e-05,1.249728
1.000000e-04,1.231377
1.000000e-03,1.188736
5.000000e-03,1.127738
1.000000e-02,1.130740
5.000000e-02,1.134417


In [144]:
# EUR
df=create_outer_stat_file(discoveries[1],target,imp,test_name,n_repetitions,n_folds,base_rep, method,save_file=False)
df[['hp', metric, f'{metric}.outer.rank', 'rep']].groupby('hp')[[metric]].mean()

,or.all
hp,
5.000000e-08,1.309862
1.000000e-07,1.308309
1.000000e-06,1.328977
1.000000e-05,1.318348
1.000000e-04,1.323218
1.000000e-03,1.300102
5.000000e-03,1.272403
1.000000e-02,1.262064
5.000000e-02,1.293959


Average of statistics results including the multi step

In [145]:
multi_discovery='+'.join(discoveries)
test_name='test.multi'

df_multi_all=create_outer_stat_file(multi_discovery,target,imp,test_name,n_repetitions,n_folds,base_rep, method,save_file=False, rank_by='or')
df_multi_all_agg=df[['hp', metric, f'{metric}.outer.rank', 'rep']].groupby('hp')[[metric]].mean()
display(df_multi_all)

,hp,beta,se,pval,prs.ngk.r2,null.ngk.r2,all.ngk.r2,prs.auroc,null.auroc,all.auroc,...,or.80.ci.min,or.80.ci.max,or.90,or.90.ci.min,or.90.ci.max,or.all.rank,or.90.rank,or.rank,rep,or.all.outer.rank
0,0.00000005+0.00000005,1.694750,0.241863,2.433756e-12,0.048074,0.048074,0.038731,0.606320,0.606320,0.055609,...,1.050847,3.219573,1.571905,0.588714,4.690448,59,144,203,1,60.0
1,0.00000005+0.0000001,1.656753,0.235625,2.046188e-12,0.048338,0.048338,0.038995,0.607681,0.607681,0.056970,...,1.097005,3.317104,1.379012,0.523820,3.888401,49,154,203,1,50.0
2,0.00000005+0.000001,1.595769,0.218894,3.096320e-13,0.051378,0.051378,0.042036,0.613484,0.613484,0.062773,...,1.071725,3.241330,1.076771,0.409555,2.925731,32,184,216,1,33.0
3,0.00000005+0.00001,1.454359,0.200954,4.577635e-13,0.050804,0.050804,0.041461,0.610306,0.610306,0.059596,...,1.053333,3.119350,2.444937,0.889837,7.931996,40,83,123,1,41.0
4,0.00000005+0.0001,1.488515,0.202542,1.994631e-13,0.052076,0.052076,0.042733,0.613418,0.613418,0.062707,...,1.017327,3.012707,1.444452,0.548696,4.072809,26,149,175,1,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,1.0+0.2,0.961010,0.194503,7.778196e-07,0.023621,0.023621,0.019054,0.579968,0.579968,0.054596,...,0.979481,2.853903,2.560678,0.931993,8.307359,151,37,188,4,151.0
192,1.0+0.3,0.947335,0.194988,1.183174e-06,0.022984,0.022984,0.018417,0.577932,0.577932,0.052560,...,0.881447,2.537290,1.584462,0.601938,4.467984,159,122,281,4,159.0
193,1.0+0.4,0.929284,0.194979,1.878428e-06,0.022279,0.022279,0.017712,0.575929,0.575929,0.050558,...,0.946054,2.756566,1.512873,0.574707,4.266605,164,132,296,4,164.0
194,1.0+0.5,0.932992,0.195837,1.896803e-06,0.022265,0.022265,0.017698,0.575804,0.575804,0.050433,...,0.933946,2.688751,1.341709,0.510422,3.644967,165,162,327,4,165.0


### New Analysis Function

In [146]:
def prs_analysis(discovery,target,imp,outer_test_name,inner_test_name,folds,reps,base_rep,method,rank_by=""):

    df_outer_all = create_outer_stat_file(discovery,target,imp,outer_test_name,reps,folds,base_rep, method)
    
    metric_suffix=""
#     fold_range=list(range(1,folds+1))
    if rank_by:
        metric_suffix=f'_by_{rank_by}'
#         fold_range=['avg']
        
    # df of statistics for inner loops over all reps:
    df_cat=pd.DataFrame()
    dfs=[]
    for rep in range(1,reps+1):
        for fold in range(1,folds+1):
            path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/rep_'+str(base_rep)+'_'+str(rep)+'/'+'prs.cv.'+method+'___'
            path=f'{path}{fold}_{folds}_{inner_test_name}.statistics{metric_suffix}.tsv'
            inner_df=pd.read_csv(path, sep='\t')
            inner_df['fold']=fold
            inner_df['rep']=rep
            dfs.append(inner_df)
    
    df_cat=pd.concat(dfs, ignore_index=True)

#     # Extract optimal hps for outer CV
#     df_outer_optimal_hps=df_cat.loc[df_cat.groupby('rep')['or.all'].idxmax(),['rep', 'fold', 'hp']]
#     display(df_outer_optimal_hps)
    
    # Check te performance of the optimal inner hps on the outer CV
    df_inner_all=pd.concat(dfs, ignore_index=True)
    df_inner_mean_or=df_inner_all.groupby(['rep', 'hp'])[[metric]].mean().reset_index()
    df_optimal_hps=df_inner_mean_or.loc[df_inner_mean_or.groupby('rep')[metric].idxmax()]

    # Check te performance of the optimal outer hps on the outer CV
    df_outer_all.index=df_outer_all.apply(lambda a: f'{a["rep"]}_{a["hp"]}', axis=1)
    result_true=df_outer_all.loc[df_optimal_hps.apply(lambda a: f'{a["rep"]}_{a["hp"]}', axis=1)]
#     print('the or.all in the outer loop using the optimal hps in the inner loop: \n(best practice)')
#     display(result_true['or.all'])
    
#     print('the max or.all in the outer loop')
#     display(df_outer_all.groupby('rep')[['or.all']].max())

    
    max_or_all = df_outer_all.groupby('rep')[metric].max()
    result_max = pd.merge(df_outer_all, max_or_all, on=['rep', metric])
# #     print('and their full info')
# #     display(result_max)

    return result_true, result_max

# prs_analysis(mono_discovery0,target,imp,'test','validation',folds,reps,base_rep, method, rank_by)

In [147]:
mono_discovery0=discoveries[0]
mono_discovery1=discoveries[1]
multi_discovery="+".join(discoveries)

folds=3
reps=4

In [148]:
# non-EUR
df_true, df_max=prs_analysis(mono_discovery0,target,imp,'test','validation',folds,reps,base_rep, method, rank_by)
df_non_eur_outer_rep=df_true[['hp', metric, f'{metric}.outer.rank', 'rep']]
display(df_non_eur_outer_rep)
print("mean:", round(df_true[metric].mean(),4))

,hp,or.all,or.all.outer.rank,rep
1.0_1e-05,0.00001,1.262377,5.0,1
2.0_1e-05,0.00001,1.139360,5.0,2
3.0_0.0001,0.00010,1.200927,5.0,3
4.0_1e-05,0.00001,1.289159,2.0,4


mean: 1.223


In [149]:
# EUR
df_true, df_max=prs_analysis(mono_discovery1,target,imp,'test','validation',folds,reps,base_rep,method, rank_by)
df_eur_outer_rep=df_true[['hp', metric, f'{metric}.outer.rank', 'rep']]
display(df_eur_outer_rep)
print("mean:", round(df_true[metric].mean(),4))

,hp,or.all,or.all.outer.rank,rep
1.0_0.2,0.200000,1.331783,10.0,1
2.0_0.0001,0.000100,1.174745,9.0,2
3.0_0.0001,0.000100,1.348234,10.0,3
4.0_1e-06,0.000001,1.304638,5.0,4


mean: 1.2898


In [150]:
# MULTI
rank_by=""
df_true, df_max=prs_analysis(multi_discovery,target,imp,'test.multi','validation.multi',folds,reps,base_rep,method)
display(df_true[['hp', metric, f'{metric}.outer.rank', 'rep']]) # .groupby('hp')[['or.all']].mean()
print("mean:", round(df_true[metric].mean(),4))

,hp,or.all,or.all.outer.rank,rep
1_0.00001+0.2,0.00001+0.2,1.425959,65.0,1
2_0.00001+0.3,0.00001+0.3,1.182284,116.0,2
3_0.0001+0.2,0.0001+0.2,1.358985,80.0,3
4_0.00000005+0.1,0.00000005+0.1,1.302102,137.0,4


mean: 1.3173


In [151]:
def extract_top_mono_hp_sets(a):
    for i, (cur_non_eur, cur_eur) in enumerate(zip(df_non_eur_outer_rep['hp'], df_eur_outer_rep['hp'])):
        if a['hp1']==cur_non_eur and a['hp2']==cur_eur and a['rep']==i+1:
            return True
    return False

hp_series=df_multi_all['hp']
df_multi_all['hp1']=hp_series.apply(lambda a : a.split('+')[0]).astype(float).values
df_multi_all['hp2']=hp_series.apply(lambda a : a.split('+')[1]).astype(float).values
df_multi_from_mono=df_multi_all[df_multi_all.apply(extract_top_mono_hp_sets, axis=1)]
display(df_multi_from_mono[['hp', metric, f'{metric}.outer.rank', 'rep']])
df_multi_from_mono[metric].mean()


,hp,or.all,or.all.outer.rank,rep
51,0.00001+0.2,1.425959,65.0,1
46,0.00001+0.0001,1.184386,112.0,2
60,0.0001+0.0001,1.332955,116.0,3
44,0.00001+0.000001,1.341612,79.0,4


1.321227996511555

In [35]:
res['hp'].to_list()

NameError: name 'res' is not defined

## Ranking the output - only in multi

In [ ]:
# # task-> create average scoring file
# inner_test_name='validation.multi'
# df1=pd.DataFrame()
# dfs=[]
# for cur_rep in range(1, 2):#n_repetitions+1):
#     for fold in range(1,n_folds+1):
#         prs_rep_path=f'{PRSS_PATH}{multi_discovery}_{target}/{imp}/rep_{base_rep}_{cur_rep}/prs.cv.{method}___'
#         rank_file=f'{prs_rep_path}{fold}_{n_folds}_{inner_test_name}_hps_all_summary.tsv'
#         df_rank=pd.read_csv(rank_file, sep='\t')
#         dfs.append(df_rank)

#     all_df = pd.concat(dfs)
#     result = all_df.groupby('hp')['score'].sum().reset_index()
#     result['score']/=n_folds
#     print(result)
# print(rank_file)

In [36]:
inner_test_name='validation.multi'
# outer_test_name='test.multi'
ranked_stat_df=pd.DataFrame()
for cur_rep in range(1, n_repetitions+1):
    prs_rep_path=f'{PRSS_PATH}{multi_discovery}_{target}/{imp}/rep_{base_rep}_{cur_rep}/prs.cv.{method}___' 
    # rank_file=f'{prs_rep_path}{n_folds}_{inner_test_name}_hps_avg_score_rank.tsv'
    avg_or_rank=f'{prs_rep_path}avg_3_validation.multi.statistics_by_or.tsv'
    df_rank=pd.read_csv(rank_file, sep='\t')
    df_rank['rep']=cur_rep
    
    ranked_stat_df=pd.concat([ranked_stat_df, df_rank])
    

display(ranked_stat_df)
# print(avg_or_rank)
# filtered_ranked=ranked_stat_df[ranked_stat_df['or.all.rank']<=10].sort_values(by=['rep', 'or.all.rank'],ascending=True)
# display(filtered_ranked)


### Numbers are 'longer', seem equal with different rank but not all is displayed

NameError: name 'rank_file' is not defined

In [ ]:
# inner_test_name='validation.multi'
# # outer_test_name='test.multi'
# ranked_stat_df=pd.DataFrame()
# for cur_rep in range(1, n_repetitions+1):
#     prs_rep_path=f'{PRSS_PATH}{multi_discovery}_{target}/{imp}/rep_{base_rep}_{cur_rep}/prs.cv.{method}___' 
#     rank_file=f'{prs_rep_path}{n_folds}_{inner_test_name}_hps_avg_score_rank.tsv'
#     df_rank=pd.read_csv(rank_file, sep='\t')
#     df_rank['rep']=cur_rep
    
#     ranked_stat_df=pd.concat([ranked_stat_df, df_rank])
    

# # display(ranked_stat_df)
# filtered_ranked=ranked_stat_df[ranked_stat_df['rank']<=10].sort_values(by=['rep', 'rank'],ascending=True)
# display(filtered_ranked)


# ### Numbers are 'longer', seem equal with different rank but not all is displayed

In [ ]:
test_name='test.multi'
inner_test_name='validation.multi'
# filtered_ranked.rename(columns={'or.all.rank':'inner_or.all.rank'},inplace=True)
for i in range(1,n_repetitions+1):
    top_ranked_lst=filtered_ranked[filtered_ranked['rep']==i]['hp'].to_list()
    prs_rep_path=f'{PRSS_PATH}{multi_discovery}_{target}/{imp}/rep_{base_rep}_{i}/prs.cv.{method}___'
    path=f'{prs_rep_path}{folds}_{test_name}.statistics_by_{rank_by}.tsv'
    df_stat=pd.read_csv(path, sep='\t')
    # df_stat.rename(columns={'or.all.rank':'outer_or.all.rank'},inplace=True)
    df_stat=df_stat.merge(filtered_ranked,on='hp',how='inner',suffixes=['_test','_validation']).sort_values(by='or.all.rank_validation')
    # display(df_stat[df_stat['hp'].isin(top_ranked_lst)][['hp','or.all','inner_rank']])

    # avg_or_rank=f'{prs_rep_path}avg_3_validation.multi.statistics_by_or.tsv'
    # df_avg_or_rank=pd.read_csv(avg_or_rank, sep='\t')
    # # display(df_avg_or_rank[['hp','or.rank']])
    # df_stat=df_stat.merge(df_avg_or_rank[['hp','or.all.rank']],on='hp',how='inner',suffixes=['_test','_validation'])

    #find the outer rank
    # outer_rank_file=f'{prs_rep_path}{n_folds}_{test_name}_hps_score_rank.tsv'
    # df_rank_outer=pd.read_csv(outer_rank_file, sep='\t')
    # df_rank_outer.rename(columns={'or.all.rank':'outer_or.all.rank'},inplace=True)
    # df_stat=df_stat.merge(df_rank_outer,on='hp',how='inner')
    # display(df_stat)
    print(f'rep {i}')
    display(df_stat[df_stat['hp'].isin(top_ranked_lst)][['hp','or.all_validation','or.all_test', 'or.all.rank_validation', 'or.all.rank_test']])



In [ ]:
# test_name='test.multi'
# inner_test_name='validation.multi'
# filtered_ranked.rename(columns={'rank':'inner_rank'},inplace=True)
# for i in range(1,n_repetitions+1):
#     top_ranked_lst=filtered_ranked[filtered_ranked['rep']==i]['hp'].to_list()
#     prs_rep_path=f'{PRSS_PATH}{multi_discovery}_{target}/{imp}/rep_{base_rep}_{i}/prs.cv.{method}___'
#     path=f'{prs_rep_path}{folds}_{test_name}.statistics_by_{rank_by}.tsv'
#     df_stat=pd.read_csv(path, sep='\t')
#     df_stat=df_stat.merge(filtered_ranked,on='hp',how='inner').sort_values(by='inner_rank')
#     # display(df_stat[df_stat['hp'].isin(top_ranked_lst)][['hp','or.all','inner_rank']])

#     avg_or_rank=f'{prs_rep_path}avg_3_validation.multi.statistics_by_or.tsv'
#     df_avg_or_rank=pd.read_csv(avg_or_rank, sep='\t')
#     # display(df_avg_or_rank[['hp','or.rank']])
#     df_stat=df_stat.merge(df_avg_or_rank[['hp','or.all.rank']],on='hp',how='inner',suffixes=['_test','_validation'])

#     #find the outer rank
#     outer_rank_file=f'{prs_rep_path}{n_folds}_{test_name}_hps_score_rank.tsv'
#     df_rank_outer=pd.read_csv(outer_rank_file, sep='\t')
#     df_rank_outer.rename(columns={'rank':'outer_rank'},inplace=True)
#     df_stat=df_stat.merge(df_rank_outer,on='hp',how='inner')
#     display(df_stat[df_stat['hp'].isin(top_ranked_lst)][['hp','or.all','inner_rank','outer_rank', 'or.all.rank_validation', 'or.all.rank_test']])



In [ ]:
avg_or_rank=f'/specific/elkon/maibendayan/PRS/PRSs/mai_bcac_onco_afr_no_1200_usa-5pcs+mai_bcac_onco_eur-5pcs_mai_bcac_onco_afr_1200_usa/impX/rep_103_1/prs.cv.pt3___avg_3_validation.multi.statistics_by_or.tsv'
df_avg_or_rank=pd.read_csv(avg_or_rank, sep='\t')
display(df_avg_or_rank[['hp','or.all.rank']])

In [ ]:
['0.1+0.000001', '0.1+1.0', '0.1+0.001', '0.00001+0.000001']

In [ ]:
# opt_hps_multi=['0.1+0.001','0.3+0.000001','0.0001+0.000001','0.1+0.001']
opt_hps_multi=['0.1+0.000001', '0.1+1.0', '0.1+0.001', '0.00001+0.000001']

ranked_opt=pd.DataFrame()
for i in range(len(opt_hps_multi)):
    ranked_opt=pd.concat([ranked_opt,ranked_stat_df[(ranked_stat_df['hp'] == opt_hps_multi[i]) & (ranked_stat_df['rep'] == i+1)]])

display(ranked_opt)

## check the new multi pipeline

In [38]:
# df_new_multi=pd.DataFrame(data={'hps':[],'combo1':[],'combo2':[],'combo3':[], 'max_three':[]})
inner_best_hps_list=[]
test_name='test.multi'
inner_test_name='validation.multi'
rank_by='or'
for cur_rep in range(1,5):
    prs_rep_path=f'{PRSS_PATH}{multi_discovery}_{target}/{imp}/rep_{base_rep}_{cur_rep}/prs.cv.{method}___'
    stat_path=f'{prs_rep_path}{folds}_{test_name}.statistics_by_{rank_by}.tsv'
    # print(stat_path)

    stat_df=pd.read_csv(stat_path, sep='\t')
    stat_df['rep']=cur_rep
#   print('all outer test:')
#   display(stat_df[['hp','or.all','rep']])

    # for cur_fold in range(1,4):
    #     print(f'inner {cur_fold}')
    #     inner_stat_path=f'{prs_rep_path}{cur_fold}_{folds}_{inner_test_name}.statistics.tsv'
    #     inner_stat_df=pd.read_csv(inner_stat_path, sep='\t')
    #     display(inner_stat_df.nlargest(1, 'or.all')[['hp','or.all']])
    inner_stat_path=f'{prs_rep_path}avg_{folds}_{inner_test_name}.statistics_by_{rank_by}.tsv'
    inner_stat_df=pd.read_csv(inner_stat_path, sep='\t')
    inner_stat_df['rep']=cur_rep
    print('inner validation')
    df_top_inner=inner_stat_df.nlargest(3, 'or.all')
    display(df_top_inner[['hp','or.all', 'or.all.rank','rep']])
    print('outer using hp from validation')
    display(stat_df[stat_df['hp'].isin(df_top_inner['hp'])][['hp','or.all', 'or.all.rank','rep']])
    inner_best_hps_list.append(inner_stat_df.nlargest(5, 'or.all')['hp'].tolist())


    # tmp['combo1']= stat_path=f'{prs_rep_path}{folds}_{test_name}.statistics_by_{rank_by}.tsv'
    
print(inner_best_hps_list)
    
###### check another pipeline idea: 
# print('\n.......................................\n')
# for cur_rep in range(1,5):
#     prs_rep_path=f'{PRSS_PATH}{multi_discovery}_{target}/{imp}/all_combos/rep_{base_rep}_{cur_rep}/prs.cv.{method}___'
#     stat_path=f'{prs_rep_path}{folds}_{test_name}.statistics_by_{rank_by}.tsv'
#     # print(stat_path)
#     stat_df=pd.read_csv(stat_path, sep='\t')
#     stat_df['rep']=cur_rep
#     filtered_df=stat_df[stat_df['hp'].isin(inner_best_hps_list[cur_rep-1])]
#     print('outer test, different pipeline')
#     display(filtered_df[['hp','or.all','rep']])


inner validation


,hp,or.all,or.all.rank,rep
2857,0.9-12+0.9-7,1.466060,1,1
2858,0.9-12+0.9-8,1.465096,2,1
2856,0.9-12+0.9-6,1.464088,3,1


outer using hp from validation


,hp,or.all,or.all.rank,rep
3485,0.9-12+0.9-6,1.693355,29,1
3486,0.9-12+0.9-7,1.694095,26,1
3487,0.9-12+0.9-8,1.693858,27,1


inner validation


...

inner validation


,hp,or.all,or.all.rank,rep
2839,0.9-12+0.5-9,1.507044,1,4
2801,0.9-12+0.2-10,1.505486,2,4
2858,0.9-12+0.9-8,1.504886,3,4


outer using hp from validation


,hp,or.all,or.all.rank,rep
3449,0.9-12+0.2-10,1.528075,143,4
3468,0.9-12+0.5-9,1.547672,110,4
3487,0.9-12+0.9-8,1.568580,78,4


[['0.9-12+0.9-7', '0.9-12+0.9-8', '0.9-12+0.9-6', '0.5-12+0.9-7', '0.5-12+0.9-8'], ['0.9-12+0.9-7', '0.9-12+0.9-8', '0.9-12+0.9-6', '0.9-12+0.9-5', '0.9-12+0.5-9'], ['0.5-12+0.9-6', '0.5-12+0.9-7', '0.5-12+0.9-5', '0.9-12+0.9-6', '0.5-12+0.9-4'], ['0.9-12+0.5-9', '0.9-12+0.2-10', '0.9-12+0.9-8', '0.9-12+0.5-10', '0.9-12+0.2-9']]


## Multi

In [39]:
r=4
test_multi='test.multi'
inner_test_name_multi='validation.multi'
displayed_fields=['hp','or.all', 'or.all.se', 'or.all.rank']
prs_rep_path=f'{PRSS_PATH}{multi_discovery}_{target}/{imp}/rep_{base_rep}_{r}/prs.cv.{method}___' # /all_combos
stat_path=f'{prs_rep_path}{folds}_{test_multi}.statistics_by_{rank_by}.tsv'
stat_df=pd.read_csv(stat_path, sep='\t')
# filtered_df=stat_df[stat_df['hp'].isin(inner_best_hps_list[cur_rep-1])]
# display(stat_df[['hp','or.all']].sort_values(by='or.all',ascending=False))

# list=stat_df.nlargest(5,'or.all')['hp'].to_list()
prs_rep_path=f'{PRSS_PATH}{multi_discovery}_{target}/{imp}/rep_{base_rep}_{r}/prs.cv.{method}___'
stat_path=f'{prs_rep_path}avg_{folds}_{inner_test_name_multi}.statistics_by_{rank_by}.tsv'
stat_inner_df=pd.read_csv(stat_path, sep='\t')

# display(stat_df[stat_df['hp'].isin(list)][['hp','or.all','or.all.rank']])
# display(stat_df.nlargest(5,'or.all')[['hp','or.all','or.all.rank']])
# display(stat_inner_df[['hp','or.all','or.all.rank']].sort_values(by='or.all',ascending=False))

test_multi='test.multi'
inner_test_name_multi='validation.multi'
prs_rep_path=f'{PRSS_PATH}{multi_discovery}_{target}/{imp}/rep_{base_rep}_{r}/prs.cv.{method}___' # /all_combos
alphas_path=f'{prs_rep_path}{folds}_{test_multi}_hps_all_summary.tsv'
alphas_df=pd.read_csv(alphas_path, sep='\t')


merged_df=stat_df[displayed_fields].merge(stat_inner_df[displayed_fields],on='hp', suffixes=('_outer','_inner'))

merged_df=merged_df.merge(alphas_df,on='hp').sort_values(by=["or.all_inner"], ascending=False)

display(merged_df)

,hp,or.all_outer,or.all.se_outer,or.all.rank_outer,or.all_inner,or.all.se_inner,or.all.rank_inner,alpha0,alpha1,score
3468,0.9-12+0.5-9,1.547672,0.095263,110,1.507044,0.093524,1,0.188003,0.078018,0.543877
3449,0.9-12+0.2-10,1.528075,0.092145,143,1.505486,0.091254,2,0.182528,0.104674,0.546770
3487,0.9-12+0.9-8,1.568580,0.096690,78,1.504886,0.093817,3,0.188776,0.069507,0.538091
3469,0.9-12+0.5-10,1.537062,0.091952,126,1.503720,0.090784,4,0.178995,0.108673,0.546770
3448,0.9-12+0.2-9,1.536760,0.095169,127,1.503235,0.093525,5,0.190792,0.068654,0.547734
...,...,...,...,...,...,...,...,...,...,...
4619,1-10+0.9-20,1.234164,0.059957,4608,1.172136,0.057932,5116,0.026463,0.041132,0.524590
4299,1-6+0.9-20,1.213344,0.061481,4914,1.171411,0.059615,5117,0.079488,0.041499,0.531340
4539,1-9+0.9-20,1.219106,0.059874,4815,1.171218,0.058355,5118,0.042219,0.041014,0.518804
4459,1-8+0.9-20,1.215909,0.060426,4881,1.170473,0.058844,5119,0.057431,0.041192,0.531340


## MONO

In [40]:
cur_rep=1
metric='or.all' #  prs.auroc
displayed_field=['hp'] + [metric] # , 'or.all.se' ,'or.all'
test_name='test'
inner_test_name='validation'

prs_rep_path=f'{PRSS_PATH}{mono_discovery0}_{target}/{imp}/rep_{base_rep}_{cur_rep}/prs.cv.{method}___'
stat_path=f'{prs_rep_path}{folds}_{test_name}.statistics_by_{rank_by}.tsv'
stat_df=pd.read_csv(stat_path, sep='\t')

# filtered_df=stat_df[stat_df['hp'].isin(inner_best_hps_list[cur_rep-1])]
# display(stat_df[displayed_field].sort_values(by='or.all',ascending=False))

avg_lst=[]
for cur_fold in range(1,4):
    # list=stat_df.nlargest(5,'or.all')['hp'].to_list()
    prs_rep_path=f'{PRSS_PATH}{mono_discovery0}_{target}/{imp}/rep_{base_rep}_{cur_rep}/prs.cv.{method}___'
    inner_stat_path=f'{prs_rep_path}{cur_fold}_{folds}_{inner_test_name}.statistics_by_{rank_by}.tsv'
    inner_stat_df=pd.read_csv(inner_stat_path, sep='\t')
    # display(inner_stat_df[stat_df['hp'].isin(list)][displayed_field])
    avg_lst.append(inner_stat_df)
avg_df=pd.concat(avg_lst)
print('average:')
# display(avg_df)
avg_df=avg_df.groupby('hp').mean()[displayed_field[1:]].sort_values(by=metric,ascending=False)
avg_df=avg_df.reset_index()
# display(stat_df.nlargest(5,'or.all')[displayed_field])


# display(stat_df[displayed_field].sort_values(by='or.all',ascending=False))

merged_df=stat_df[displayed_field].merge(avg_df[displayed_field],on='hp', suffixes=('_outer','_inner'))

# merged_df=merged_df.merge(alphas_df,on='hp').sort_values(by=["or.all_inner"], ascending=False)

merged_df[f'{metric}.rank_inner']=len(merged_df[f"{metric}_inner"])-merged_df[f"{metric}_inner"].rank()+1
merged_df[f'{metric}.rank_outer']=len(merged_df[f"{metric}_outer"])-merged_df[f"{metric}_outer"].rank()+1

for i, cur_df in enumerate(avg_lst):
    cur_df['fold']=i+1
    merged_df=merged_df.merge(cur_df[displayed_field], on='hp') # ,suffixes=('',f'{i+1}')
    merged_df=merged_df.rename(columns={metric: f'{metric}_{i+1}'})
    merged_df[f'{metric}.rank_{i+1}']=len(merged_df[f'{metric}_{i+1}'])-merged_df[f'{metric}_{i+1}'].rank()+1
    
merged_df['rank_inner']=merged_df[[f'{metric}.rank_{a}' for a in range(1,len(avg_lst)+1)]].mean(axis=1)
# display(merged_df.sort_values(by=[f"{metric}_inner"], ascending=False)) # 
display(merged_df.sort_values(by=['rank_inner'], ascending=True)) # 

average:


,hp,or.all_outer,or.all_inner,or.all.rank_inner,or.all.rank_outer,or.all_1,or.all.rank_1,or.all_2,or.all.rank_2,or.all_3,or.all.rank_3,rank_inner
11,0.2-12,1.399932,1.288284,2.0,3.0,1.242167,5.0,1.226816,15.0,1.395868,1.0,7.000000
26,0.5-12,1.411930,1.290327,1.0,1.0,1.245298,4.0,1.233467,14.0,1.392217,3.0,7.000000
41,0.9-12,1.406916,1.281992,3.0,2.0,1.234543,7.0,1.240074,12.0,1.371359,6.0,8.333333
24,0.5-10,1.267288,1.260244,8.0,21.0,1.224061,8.0,1.181228,25.0,1.375444,5.0,12.666667
9,0.2-10,1.249787,1.254896,9.0,23.0,1.215384,10.0,1.190669,22.0,1.358636,9.0,13.666667
23,0.5-9,1.214341,1.241460,15.0,30.0,1.205009,15.0,1.195429,20.0,1.323942,11.0,15.333333
7,0.2-8,1.188138,1.234781,18.0,51.0,1.208188,11.0,1.210622,17.0,1.285531,20.0,16.000000
25,0.5-11,1.361693,1.270208,4.0,6.0,1.250822,1.0,1.164203,47.0,1.395598,2.0,16.666667
22,0.5-8,1.199355,1.231053,20.0,42.0,1.205740,13.0,1.196773,19.0,1.290647,18.0,16.666667
10,0.2-11,1.354197,1.269177,5.0,9.0,1.245822,3.0,1.169548,43.0,1.392163,4.0,16.666667



## MONO for original data

In [42]:
# cross validation parameters
n_folds=3
n_repetitions=4
base_rep=102


cur_rep=4
metric='or.all' #  prs.auroc
displayed_field=['hp'] + [metric] # , 'or.all.se' ,'or.all'
test_name='test'
inner_test_name='validation'

mono_discovery1='bcac_onco_eur-5pcs'
target='bcac_onco_eas2'

prs_rep_path=f'{PRSS_PATH}{mono_discovery1}_{target}/{imp}/rep_{base_rep}_{cur_rep}/prs.cv.{method}___'
stat_path=f'{prs_rep_path}{folds}_{test_name}.statistics.tsv'
stat_df=pd.read_csv(stat_path, sep='\t')

# filtered_df=stat_df[stat_df['hp'].isin(inner_best_hps_list[cur_rep-1])]
# display(stat_df[displayed_field].sort_values(by='or.all',ascending=False))

avg_lst=[]
for cur_fold in range(1,4):
    # list=stat_df.nlargest(5,'or.all')['hp'].to_list()
    prs_rep_path=f'{PRSS_PATH}{mono_discovery1}_{target}/{imp}/rep_{base_rep}_{cur_rep}/prs.cv.{method}___'
    inner_stat_path=f'{prs_rep_path}{cur_fold}_{folds}_{inner_test_name}.statistics.tsv'
    inner_stat_df=pd.read_csv(inner_stat_path, sep='\t')
    # display(inner_stat_df[stat_df['hp'].isin(list)][displayed_field])
    avg_lst.append(inner_stat_df)
avg_df=pd.concat(avg_lst)
print('average:')
# display(avg_df)
avg_df=avg_df.groupby('hp').mean()[displayed_field[1:]].sort_values(by=metric,ascending=False)
avg_df=avg_df.reset_index()
# display(stat_df.nlargest(5,'or.all')[displayed_field])


# display(stat_df[displayed_field].sort_values(by='or.all',ascending=False))

merged_df=stat_df[displayed_field].merge(avg_df[displayed_field],on='hp', suffixes=('_outer','_inner'))

# merged_df=merged_df.merge(alphas_df,on='hp').sort_values(by=["or.all_inner"], ascending=False)

merged_df[f'{metric}.rank_inner']=len(merged_df[f"{metric}_inner"])-merged_df[f"{metric}_inner"].rank()+1
merged_df[f'{metric}.rank_outer']=len(merged_df[f"{metric}_outer"])-merged_df[f"{metric}_outer"].rank()+1

for i, cur_df in enumerate(avg_lst):
    cur_df['fold']=i+1
    merged_df=merged_df.merge(cur_df[displayed_field], on='hp') # ,suffixes=('',f'{i+1}')
    merged_df=merged_df.rename(columns={metric: f'{metric}_{i+1}'})
    merged_df[f'{metric}.rank_{i+1}']=len(merged_df[f'{metric}_{i+1}'])-merged_df[f'{metric}_{i+1}'].rank()+1
    
merged_df['rank_inner']=merged_df[[f'{metric}.rank_{a}' for a in range(1,len(avg_lst)+1)]].mean(axis=1)
display(merged_df.sort_values(by=[f"{metric}_inner"], ascending=False))# f"{metric}_inner"

average:


,hp,or.all_outer,or.all_inner,or.all.rank_inner,or.all.rank_outer,or.all_1,or.all.rank_1,or.all_2,or.all.rank_2,or.all_3,or.all.rank_3,rank_inner
48,0.9-9,1.591683,1.646650,1.0,6.0,1.791092,5.0,1.425693,1.0,1.723166,1.0,2.333333
47,0.9-8,1.600214,1.643767,2.0,4.0,1.799941,3.0,1.422535,2.0,1.708826,3.0,2.666667
29,0.5-10,1.523736,1.637176,3.0,12.0,1.803372,2.0,1.389395,6.0,1.718760,2.0,3.333333
28,0.5-9,1.515171,1.636256,4.0,14.0,1.827028,1.0,1.378931,7.0,1.702811,4.0,4.000000
46,0.9-7,1.607943,1.622952,5.0,2.0,1.778245,6.0,1.414352,3.0,1.676259,7.0,5.333333
...,...,...,...,...,...,...,...,...,...,...,...,...
78,1-19,1.151780,1.136658,76.0,74.0,1.184502,76.0,1.041216,75.0,1.184257,76.0,75.666667
19,0.2-20,1.141146,1.123969,77.0,80.0,1.142567,80.0,1.018192,80.0,1.211149,67.0,75.666667
39,0.5-20,1.142588,1.121172,78.0,79.0,1.144568,79.0,1.019023,79.0,1.199926,72.0,76.666667
79,1-20,1.144144,1.119324,79.0,77.0,1.156427,77.0,1.026210,77.0,1.175336,78.0,77.333333


In [58]:
all_path=f'/home/gaga/gaga-pd/prs_data/datasets/dec/bcac_onco_eas/pheno'
all_df=pd.read_csv(all_path, sep='\t')
all_df.index=all_df.iloc[:,0]
for a in range(1,5):
    
    
#     path=f'/home/gaga/gaga-pd/prs_data/datasets/dec/bcac_onco_eas/rep_102_{a}/impX/ds___3_test.fam'
#     df=pd.read_csv(path, sep='\t', header=None)
#     ids=df.iloc[:,0]
#     df=all_df.loc[ids]
#     path=f'/home/gaga/gaga-pd/prs_data/datasets/dec/bcac_onco_eas/rep_102_{a}/pheno___3_test'
#     df.to_csv(path, sep='\t', index=False)
    
    path=f'/home/gaga/gaga-pd/prs_data/datasets/dec/bcac_onco_eas/rep_102_{a}/impX/ds___3_both.fam'
    df=pd.read_csv(path, sep='\t', header=None)
    ids=df.iloc[:,0]
    df=all_df.loc[ids]
    path=f'/home/gaga/gaga-pd/prs_data/datasets/dec/bcac_onco_eas/rep_102_{a}/pheno___3_both'
    df.to_csv(path, sep='\t', index=False)
    
#     for b in range (1,4):
        
#         path=f'/home/gaga/gaga-pd/prs_data/datasets/dec/bcac_onco_eas/rep_102_{a}/impX/ds___{b}_3_validation.fam'
#         df=pd.read_csv(path, sep='\t', header=None)
#         ids=df.iloc[:,0]
#         df=all_df.loc[ids]
#         path=f'/home/gaga/gaga-pd/prs_data/datasets/dec/bcac_onco_eas/rep_102_{a}/pheno___{b}_3_validation'
#         df.to_csv(path, sep='\t', index=False)
        
#         path=f'/home/gaga/gaga-pd/prs_data/datasets/dec/bcac_onco_eas/rep_102_{a}/impX/ds___{b}_3_train.fam'
#         df=pd.read_csv(path, sep='\t', header=None)
#         ids=df.iloc[:,0]
#         df=all_df.loc[ids]
#         path=f'/home/gaga/gaga-pd/prs_data/datasets/dec/bcac_onco_eas/rep_102_{a}/pheno___{b}_3_train'
#         df.to_csv(path, sep='\t', index=False)
        

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['FID'], dtype='object', name='FID'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [44]:
path=f'/home/gaga/gaga-pd/prs_data/datasets/dec/bcac_onco_eas2/pheno'
df=pd.read_csv(path, sep='\t')
df["label"]=df["label"].apply(lambda a: int(a>1)+1)
df.to_csv(path, sep='\t', index=False)

In [45]:
import matplotlib.pyplot as plt
import itertools

ds='ds.1e-06' # .no_both1
pca_path=os.path.join(DATASETS_PATH, target, imp, f'{ds}.eigenvec')
ds_pca=pd.read_csv(pca_path, delim_whitespace=True, header=None)

pcs=4
combs=[a for a in itertools.combinations(range(pcs), 2)]
r=2
c=len(combs)//2
fig,axs=plt.subplots(r,c,figsize=(c*7,r*7))

d_reps={}
for r in [2]: #range(1,5):
    test_sample_path=os.path.join(DATASETS_PATH, target, f'rep_{base_rep}_{r}', f'pheno___{folds}_test')
    df_test_sample=pd.read_csv(test_sample_path, delim_whitespace=True)
    print(f'{(df_test_sample["label"]==1).sum()}/{len(df_test_sample)}')
    test_sample_ids=df_test_sample.iloc[:,0]
    d_reps.update({a.iloc[0]:r*(9+a.loc['label']) for i, a in df_test_sample.iterrows()})
    for f in range(1,4):
        val_sample_path=os.path.join(DATASETS_PATH, target, f'rep_{base_rep}_{r}', f'pheno___{f}_{folds}_validation')
        df_val_sample=pd.read_csv(val_sample_path, delim_whitespace=True)
        print(f'{(df_val_sample["label"]==1).sum()}/{len(df_val_sample)}')
        val_sample_ids=df_val_sample.iloc[:,0]
        d_reps.update({a.iloc[0]:int(f==1)*(2*f+(a.loc['label']-1)) for i, a in df_val_sample.iterrows()})
    

cmap = plt.get_cmap('Dark2')
for a, cur_comb in enumerate(combs):
    i=a//c
    j=a%c
    for f in pd.Series(list(d_reps.values())).unique():
        cur_ids=[a for a in ds_pca.iloc[:,0] if d_reps[a]==f]
        # print(len(ds_pca.iloc[ds_pca.iloc[:,0]==cur_ids,2+cur_comb[0]]))
        # print(len(ds_pca.iloc[ds_pca.iloc[:,0]==cur_ids,2+cur_comb[1]]))
        axs[i][j].scatter(ds_pca.loc[ds_pca.iloc[:,0].isin(cur_ids)].iloc[:,2+cur_comb[0]], ds_pca.loc[ds_pca.iloc[:,0].isin(cur_ids)].iloc[:,2+cur_comb[1]], color=cmap(f), label=f)
    axs[i][j].set_xlabel(f'pca {cur_comb[0]+1}')
    axs[i][j].set_ylabel(f'pca {cur_comb[1]+1}')
    axs[i][j].legend()    

# plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/home/gaga/gaga-pd/prs_data/datasets/dec/bcac_onco_eas2/impX/ds.1e-06.eigenvec'

## From here it's old

In [ ]:
df_outer_all=df.copy()
df_outer_all
# df_outer_all=df.groupby('hp')['or.all'].mean().to_frame().sort_values('or.all', ascending=False)
# df_outer_all

In [ ]:
folds=3
reps=4
base_rep=103
df_cat=pd.DataFrame()
dfs=[]
rank_by='or'
for rep in range(1,reps+1):
    for fold in range(1,folds+1):
        path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/rep_'+str(base_rep)+'_'+str(rep)+'/'+'prs.cv.'+method+'___'
        path=f'{path}{fold}_{folds}_validation.multi.statistics_by_{rank_by}.tsv'
        inner_df=pd.read_csv(path, sep='\t') # , header=None)
        inner_df['fold']=fold
        inner_df['rep']=rep
        dfs.append(inner_df)

df_cat=pd.concat(dfs, ignore_index=True)
# display(df_cat)
df_outer_optimal_hps=df_cat.loc[df_cat.groupby('rep')['or.all'].idxmax(),['rep', 'fold', 'hp']]
df_outer_optimal_hps

#### MONO EUR

In [ ]:
print(discoveries[1])

In [ ]:
discovery=discoveries[1]
test_name='test'
df_outer_all_avg=pd.DataFrame()
for cur_rep in range(1, n_repetitions+1):
    prs_rep_path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/rep_'+str(base_rep)+'_'+str(cur_rep)+'/'+'prs.cv.'+method+'___'
    # file_name=prs_rep_path+str(n_folds)+'_'+test_name+'.statistics.tsv'
    file_name=prs_rep_path+str(n_folds)+'_'+test_name+'.statistics_by_'+rank_by+'.tsv'
    df_stat=pd.read_csv(file_name, sep='\t')
    df_stat['rep']=cur_rep
    # df_stat.set_index('hp', inplace = True)
    # print(df_stat.head())
    df_outer_all_avg=pd.concat([df_outer_all_avg,df_stat])
    
output_path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/prs.cv.'+method+'___'
# df.to_csv(output_path+'all_'+str(n_repetitions)+'_'+test_name+'.statistics.tsv',mode="w", sep='\t')
# df.to_csv(output_path+'all_'+str(n_repetitions)+'_'+test_name+'.statistics_by_'+ranking_by+'.tsv',mode="w", sep='\t')
print(df_outer_all_avg)
display(df_outer_all_avg)
# print ("the next file was created: "+output_path+'all_'+str(n_repetitions)+'_'+test_name+'.statistics_by_'+ranking_by+'.tsv')

In [ ]:
mono_discovery='mai_bcac_onco_eur-5pcs' #'mai_bcac_onco_afr_no_1200_usa-5pcs' #
folds=3
reps=4
base_rep=103
df_cat=pd.DataFrame()
dfs=[]
rank_by='or'
for rep in range(1,reps+1):
    for fold in range(1,folds+1):
        path=PRSS_PATH+mono_discovery+'_'+target+'/'+imp+'/rep_'+str(base_rep)+'_'+str(rep)+'/'+'prs.cv.'+method+'___'
        path=f'{path}{fold}_{folds}_validation.statistics_by_{rank_by}.tsv'
        inner_df=pd.read_csv(path, sep='\t') # , header=None)
        inner_df['fold']=fold
        inner_df['rep']=rep
        dfs.append(inner_df)

df_inner_all=pd.concat(dfs, ignore_index=True)
df_inner_mean_or=df_inner_all.groupby(['rep', 'hp'])[['or.all']].mean().reset_index()
df_optimal_hps=df_inner_mean_or.loc[df_inner_mean_or.groupby('rep')['or.all'].idxmax()]


In [ ]:
# df_outer_all=df_outer_all_avg.groupby('hp')['or.all'].mean().to_frame().sort_values('or.all', ascending=False)

df_outer_all_avg.index=df_outer_all_avg.apply(lambda a: f'{a["rep"]}_{a["hp"]}', axis=1)

display(df_outer_all_avg.loc[df_optimal_hps.apply(lambda a: f'{a["rep"]}_{a["hp"]}', axis=1), ['or.all']])
display(df_outer_all_avg.groupby('rep')[['or.all']].max())

#### MONO AFR

In [ ]:
discovery=discoveries[0]
print(discovery)
test_name='test'
df_outer_all_avg=pd.DataFrame()
for cur_rep in range(1, n_repetitions+1):
    prs_rep_path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/rep_'+str(base_rep)+'_'+str(cur_rep)+'/'+'prs.cv.'+method+'___'
    # file_name=prs_rep_path+str(n_folds)+'_'+test_name+'.statistics.tsv'
    file_name=prs_rep_path+str(n_folds)+'_'+test_name+'.statistics_by_'+ranking_by+'.tsv'
    df_stat=pd.read_csv(file_name, sep='\t')
    df_stat['rep']=cur_rep
    # df_stat.set_index('hp', inplace = True)
    # print(df_stat.head())
    df_outer_all_avg=pd.concat([df_outer_all_avg,df_stat])
    
output_path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/prs.cv.'+method+'___'
# df.to_csv(output_path+'all_'+str(n_repetitions)+'_'+test_name+'.statistics.tsv',mode="w", sep='\t')
# df.to_csv(output_path+'all_'+str(n_repetitions)+'_'+test_name+'.statistics_by_'+ranking_by+'.tsv',mode="w", sep='\t')
print(df_outer_all_avg)
display(df_outer_all_avg)
# print ("the next file was created: "+output_path+'all_'+str(n_repetitions)+'_'+test_name+'.statistics_by_'+ranking_by+'.tsv')

In [ ]:
mono_discovery=discovery #'mai_bcac_onco_afr_no_1200_usa-5pcs' #
folds=3
reps=4
base_rep=103
df_cat=pd.DataFrame()
dfs=[]
rank_by='or'
for rep in range(1,reps+1):
    for fold in range(1,folds+1):
        path=PRSS_PATH+mono_discovery+'_'+target+'/'+imp+'/rep_'+str(base_rep)+'_'+str(rep)+'/'+'prs.cv.'+method+'___'
        path=f'{path}{fold}_{folds}_validation.statistics_by_{rank_by}.tsv'
        inner_df=pd.read_csv(path, sep='\t') # , header=None)
        inner_df['fold']=fold
        inner_df['rep']=rep
        dfs.append(inner_df)

df_inner_all=pd.concat(dfs, ignore_index=True)
df_inner_mean_or=df_inner_all.groupby(['rep', 'hp'])[['or.all']].mean().reset_index()
df_optimal_hps=df_inner_mean_or.loc[df_inner_mean_or.groupby('rep')['or.all'].idxmax()]


In [ ]:
# df_outer_all=df_outer_all_avg.groupby('hp')['or.all'].mean().to_frame().sort_values('or.all', ascending=False)

df_outer_all_avg.index=df_outer_all_avg.apply(lambda a: f'{a["rep"]}_{a["hp"]}', axis=1)

display(df_outer_all_avg.loc[df_optimal_hps.apply(lambda a: f'{a["rep"]}_{a["hp"]}', axis=1), ['or.all']])
display(df_outer_all_avg.groupby('rep')[['or.all']].max())

#### MULTI EUR AFR

In [ ]:
discovery="+".join(discoveries)
print(discovery)
test_name='test.multi'
df_outer_all_avg=pd.DataFrame()
for cur_rep in range(1, n_repetitions+1):
    prs_rep_path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/rep_'+str(base_rep)+'_'+str(cur_rep)+'/'+'prs.cv.'+method+'___'
    # file_name=prs_rep_path+str(n_folds)+'_'+test_name+'.statistics.tsv'
    file_name=prs_rep_path+str(n_folds)+'_'+test_name+'.statistics_by_'+ranking_by+'.tsv'
    df_stat=pd.read_csv(file_name, sep='\t')
    df_stat['rep']=cur_rep
    # df_stat.set_index('hp', inplace = True)
    # print(df_stat.head())
    df_outer_all_avg=pd.concat([df_outer_all_avg,df_stat])
    
output_path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/prs.cv.'+method+'___'
# df.to_csv(output_path+'all_'+str(n_repetitions)+'_'+test_name+'.statistics.tsv',mode="w", sep='\t')
# df.to_csv(output_path+'all_'+str(n_repetitions)+'_'+test_name+'.statistics_by_'+ranking_by+'.tsv',mode="w", sep='\t')
print(df_outer_all_avg)
display(df_outer_all_avg)
# print ("the next file was created: "+output_path+'all_'+str(n_repetitions)+'_'+test_name+'.statistics_by_'+ranking_by+'.tsv')

In [ ]:
mono_discovery=discovery #'mai_bcac_onco_afr_no_1200_usa-5pcs' #
folds=3
reps=4
base_rep=103
df_cat=pd.DataFrame()
dfs=[]
# test_name=validat
rank_by='or'
for rep in range(1,reps+1):
    for fold in range(1,folds+1):
        path=PRSS_PATH+mono_discovery+'_'+target+'/'+imp+'/rep_'+str(base_rep)+'_'+str(rep)+'/'+'prs.cv.'+method+'___'
        path=f'{path}{fold}_{folds}_validation.multi.statistics_by_{rank_by}.tsv'
        inner_df=pd.read_csv(path, sep='\t') # , header=None)
        inner_df['fold']=fold
        inner_df['rep']=rep
        # inner_df['rank']=len(inner_df['or.all'])-inner_df['or.all'].rank()+1
        # inner_df['distance']=inner_df['or.all'].max()-inner_df['or.all']
        dfs.append(inner_df)

df_inner_all=pd.concat(dfs, ignore_index=True)
df_inner_mean_or=df_inner_all.groupby(['rep', 'hp'])[['or.all']].mean().reset_index()
df_optimal_hps=df_inner_mean_or.loc[df_inner_mean_or.groupby('rep')['or.all'].idxmax()]


In [ ]:
pd.options.mode.chained_assignment = None  
pd.options.display.max_rows = 1850

df_inner_mean=df_inner_all[['hp','or.all','rep','fold']].groupby(['rep', 'hp'])[['or.all']].mean().reset_index()

df_inner_mean['rank']=None
df_inner_mean['distance']=None
for i in range(1,5):
    df_inner_mean['rank'].loc[df_inner_mean['rep']==i]=df_inner_mean['or.all'].loc[df_inner_mean['rep']==i].rank()
    df_inner_mean['rank'].loc[df_inner_mean['rep']==i]*=-1
    df_inner_mean['rank'].loc[df_inner_mean['rep']==i]+=len(df_inner_mean['or.all'].loc[df_inner_mean['rep']==i])+1
    df_inner_mean['distance'].loc[df_inner_mean['rep']==i]=df_inner_mean['or.all'].loc[df_inner_mean['rep']==i].max()
    df_inner_mean['distance'].loc[df_inner_mean['rep']==i]-=df_inner_mean['or.all'].loc[df_inner_mean['rep']==i]
    

# display(df_inner_mean)


display(df_inner_mean.loc[df_inner_mean['rank']<11])

display(df_inner_mean.loc[df_inner_mean.groupby('rep')['or.all'].idxmax(), ['hp', 'or.all', 'rank', 'distance']]) # 

display(df_inner_mean[df_inner_mean['hp']=='0.1+0.000001'])
display(df_inner_mean[df_inner_mean['hp']=='0.1+0.05'])


In [ ]:
# df_outer_all=df_outer_all_avg.groupby('hp')['or.all'].mean().to_frame().sort_values('or.all', ascending=False)

df_outer_all_avg.index=df_outer_all_avg.apply(lambda a: f'{a["rep"]}_{a["hp"]}', axis=1)

display(df_outer_all_avg.loc[df_optimal_hps.apply(lambda a: f'{a["rep"]}_{a["hp"]}', axis=1), ['or.all']])
display(df_outer_all_avg.groupby('rep')[['or.all']].max())
display(df_outer_all_avg.loc[[f'{rep}_{hp_afr}+{hp_eur}' for rep, hp_eur, hp_afr in [(1,"0.000001", 0.1), (2,"0.000001", 0.1), (3,"0.000001", 0.1), (4,"0.05", 0.1)]], 'or.all'])

In [ ]:
#something else

In [ ]:
# Print each rep:
cur_rep=1
prs_rep_path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/rep_'+str(base_rep)+'_'+str(cur_rep)+'/'+'prs.cv.'+method+'___'
file_name=prs_rep_path+str(n_folds)+'_'+test_name+'.statistics.tsv'
df_stat=pd.read_csv(file_name, sep='\t')
print(df_stat)

In [ ]:
# Print all reps, concatinated
# discovery_array=[multi_discovery, discoveries[0], discoveries[1]]
method='ls'
discovery_array=[multi_discovery]
for discovery in discovery_array:
    # file=PRSS_PATH+discovery+'_'+target+'/'+imp+'/prs.cv.'+method+'___'+'all_'+str(n_repetitions)+'_'+test_name+'.statistics_by_'+ranking_by+'.tsv2'
    file=PRSS_PATH+discovery+'_'+target+'/'+imp+'/prs.cv.'+method+'___'+'all_'+str(n_repetitions)+'_'+test_name+'.statistics.tsv'
    df=pd.read_csv(file, sep='\t')
    print( 'discovery : '+discovery)
    df.sort_values(by=['hr.exp.beta'], axis=0, ascending=False, inplace=True)
    pd.set_option('display.max_rows', None)
    display(df[['rep','hp','or.all', 'or.90']]) #, 'hr.exp.beta', 'hr.90'
    print("")

In [ ]:
def create_avg_stat_file():
    #read statistics_file for each fold in cur rep into dataframe
    prs_rep_path='/specific/elkon/maibendayan/PRS/PRSs/mai_bcac_onco_eur-5pcs+mai_cimba_eur_brca1_oncoarray-5pcs_mai_cimba_eur_brca1_icogs/impX_gen_all/rep_103_3/prs.cv.ls___'
    df=pd.DataFrame()
    for fold in range(1,4):
        file_name=prs_rep_path+str(fold)+'_3_validation.multi.statistics_by_hr.tsv'
        df_stat=pd.read_csv(file_name, sep='\t')
        # df_stat.set_index('hp', inplace = True)
        # print(df_stat['hr.exp.beta'])
        # sum the results
        df = pd.concat([df, df_stat], axis=0)
        # df=df.add(df_stat, fill_value=0)
    
    #create the average statistics file:
    df = df.groupby('hp').mean().reset_index()
    df.set_index('hp', inplace = True)
    # df=df/3
    
    print(df['hr.exp.beta'].head(60))
    return

create_avg_stat_file()

In [ ]:
discovery=multi_discovery
prs_rep_path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/rep_'+str(base_rep)+'_'+str(3)+'/'+'prs.cv.'+method+'___'
file_name=prs_rep_path+str(n_folds)+'_'+test_name+'.statistics'+'.tsv'
out = pd.read_csv(file_name, sep='\t')
pd.set_option('display.max_rows', None)
print(out[['hp','hr.exp.beta']].sort_values(by='hr.exp.beta',ascending=False))


# discovery=multi_discovery # #discoveries[1] #
# df=pd.DataFrame()
# for cur_rep in range(1, n_repetitions+1):
#     prs_rep_path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/rep_'+str(base_rep)+'_'+str(cur_rep)+'/'+'prs.cv.'+method+'___'
#     file_name=prs_rep_path+str(n_folds)+'_'+test_name+'.statistics_by_'+ranking_by+'.tsv'
#     df_stat=pd.read_csv(file_name, sep='\t')
#     # df_stat.set_index('hp', inplace = True)
#     # print(df_stat.head())
#     df=pd.concat([df,df_stat])
    
# output_path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/prs.cv.'+method+'___'
# df.to_csv(output_path+'all_'+str(n_repetitions)+'_'+test_name+'.statistics_by_'+ranking_by+'.tsv2',mode="w", sep='\t')
# print(df)
# print ("the next file was created: "+output_path+'all_'+str(n_repetitions)+'_'+test_name+'.statistics_by_'+ranking_by+'.tsv2')

In [ ]:
for cur_rep in range(1,5):
    prs_rep_path=f'{PRSS_PATH}{multi_discovery}_{target}/{imp}/rep_{base_rep}_{cur_rep}/prs.cv.{method}___' 
    stat_file=f'{prs_rep_path}3_test.multi.statistics_by_or.tsv'
    df_stat=pd.read_csv(stat_file,sep='\t')
    display(df_stat[['hp','or.all']])